In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

#intrinsic params
focal = 910
width = 1164
height = 874
center_x = width/2
center_y = height/2
dir = "0"
start_frame = 0
end_frame = 100 #note, when dir 4, change to 1195
total_frames = 1200 if(int(dir) != 4) else 1196


whole_video = True # for submission purposes, adds results for the last frame to make sure array is the correct size.
train = True if (int(dir) <= 4) else False # for plotting purposes later. True if it is in the train set and has ground truth labels


# Intrinsic Camera Matrix
K = np.array([[focal, 0    , center_x],
              [0    , focal, center_y],
              [0    , 0    , 1       ]])


'''
Reads and returns image frame using OpenCV given its respective video and frame number
'''
def read_specific_frame(video_directory, frame_number):
    frame_path = f"./{video_directory}/{frame_number}.jpg"

    # Check if the frame exists
    if not os.path.exists(frame_path):
        print(f"Frame {frame_number} does not exist in the directory.")
        return None

    frame = cv2.imread(frame_path)

    return frame

'''
Returns window to use for calculating the moving average of data in the dataset
'''

def getWindow(index, window_size):
    numLeft = window_size//2
    numRight = window_size//2

    if(index - numLeft <= 0):
        return 0, window_size
    if(index + numRight > end_frame-start_frame):
        return end_frame-start_frame-window_size, end_frame-start_frame
    return index-numLeft, index+numRight

'''
Creates a mask of all pixels that are completely black. 
Images have been preprocessed to crop out cars and the front hood as they mess up Essential Matrix calculations.
'''
def create_black_pixel_mask(image, threshold=0):
    black_pixels = (image == 0)
    kernel = np.ones((threshold*2+1, threshold*2+1), np.uint8)
    mask = cv2.dilate(black_pixels.astype(np.uint8), kernel, iterations=1)
    return mask


'''
Checks if pixel is within a mask that is inputted
'''
def is_near_black_pixel(point, mask):
    x, y = int(point[0]), int(point[1])
    return mask[y, x] == 1
'''
Calculates Epipole location in image from a given fundamental matrix
'''
def calculateEpipole(F):
    U, S, Vt = cv2.SVDecomp(F)

    epipole = Vt.T[:, -1]

    epipole = epipole / epipole[-1]

    epipole_x, epipole_y = int(epipole[0]), int(epipole[1])

    return epipole_x, epipole_y


In [3]:
sift = cv2.SIFT_create(contrastThreshold = 0.01, edgeThreshold=100, sigma=1.6)
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
yaw_values = []
pitch_values = []
count = 0



first_frame = read_specific_frame(dir, 0)
first_gray  = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
keypoints, descriptors = sift.detectAndCompute(first_gray, None)

second_matches = (keypoints, descriptors)
for i in range(start_frame, end_frame):
    #rame1 = read_specific_frame(dir, i)
    frame2 = read_specific_frame(dir, i+1)
    #gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    #mask1 = create_black_pixel_mask(gray1)
    #mask2 = create_black_pixel_mask(gray2)

    keypoints1, descriptors1 = second_matches
    keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)
    second_matches = (keypoints2, descriptors2)

    matches = bf.knnMatch(descriptors1,descriptors2,k=2)

    #lowes ratio
    filtered_matches = []
    ratio_thresh = 0.75
    for m,n in matches:
        if m.distance < ratio_thresh * n.distance:
            filtered_matches.append(m)
    matches = filtered_matches

    #filter based on preprocessed mask, no moving cars and car hood
    # filtered_matches = []
    # for m in matches:
    #     pt1 = keypoints1[m.queryIdx].pt
    #     pt2 = keypoints2[m.trainIdx].pt

    #     (x1, y1) = pt1
    #     (x2, y2) = pt2

    #     #if not (is_near_black_pixel(pt1, mask1) or is_near_black_pixel(pt2, mask2)):
    #     filtered_matches.append(m)
    # matches = filtered_matches

    pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])

    E, mask = cv2.findEssentialMat(pts1, pts2, K, cv2.RANSAC, prob=0.99999, threshold=0.5)
    _, R, t, mask = cv2.recoverPose(E,pts1,pts2,K)
    #r1, r2, t = cv2.decomposeEssentialMat(E)
    if(t[2] < 0):
        count += 1
    print(f"percent: {count/(i+1)}")
    print(t)
    # F = np.linalg.inv(K).T @ E @ np.linalg.inv(K)

    # if F is None:
    #     yaw_values.append(yaw)
    #     pitch_values.append(pitch)
    #     continue

    # epipole_x, epipole_y = calculateEpipole(F)
    
    # yaw = np.arctan((epipole_x - center_x) / focal)
    # pitch = np.arctan((center_y - epipole_y) / focal)

    # yaw_values.append(yaw)
    # pitch_values.append(pitch)
# if(whole_video):
#     while(end_frame < total_frames):
#         yaw_values.append(yaw)
#         pitch_values.append(pitch)
#         end_frame += 1


percent: 1.0
[[-0.04341211]
 [ 0.03842861]
 [-0.9983179 ]]
percent: 1.0
[[-0.03241933]
 [ 0.03147206]
 [-0.99897873]]
percent: 1.0
[[-0.05077947]
 [ 0.05492996]
 [-0.99719815]]
percent: 1.0
[[-0.03663052]
 [ 0.03081557]
 [-0.99885365]]
percent: 1.0
[[-0.00666658]
 [ 0.04225099]
 [-0.99908479]]
percent: 1.0
[[-0.00106469]
 [ 0.04714339]
 [-0.99888756]]
percent: 1.0
[[-0.01025121]
 [ 0.02460349]
 [-0.99964473]]
percent: 1.0
[[-0.0298971 ]
 [ 0.03752395]
 [-0.9988484 ]]
percent: 1.0
[[-0.04135273]
 [ 0.02818754]
 [-0.99874692]]
percent: 1.0
[[-0.00298992]
 [ 0.01777732]
 [-0.9998375 ]]
percent: 1.0
[[-0.06285662]
 [ 0.0095598 ]
 [-0.99797678]]
percent: 1.0
[[-0.0228073 ]
 [ 0.02899756]
 [-0.99931925]]
percent: 1.0
[[-0.09371803]
 [ 0.01828642]
 [-0.99543083]]
percent: 1.0
[[-0.05539371]
 [ 0.0389001 ]
 [-0.99770653]]
percent: 1.0
[[-0.01974848]
 [ 0.04864261]
 [-0.998621  ]]
percent: 1.0
[[-0.04394387]
 [ 0.03008138]
 [-0.99858102]]
percent: 1.0
[[-0.01911018]
 [ 0.02256548]
 [-0.9995627 